In [1]:
import numpy as np

from math import exp, sqrt
from numpy.linalg import norm

Gradient Descent

Minimize function E(u, v) = (u$e^{v}$-2v$e^{-u}$)$^{2}$
<br>
Derivatives:
<br>
$\frac{\partial E}{\partial u}$ = 2 (u$e^{v}$-2v$e^{-u}$)($e^{v}$+2v$e^{-u}$)
<br>
$\frac{\partial E}{\partial v}$ = 2 (u$e^{v}$-2v$e^{-u}$)(u$e^{v}$-2$e^{-u}$)
<br>
Start at (u,v) = (1,1) and with learning rate $\eta$=0.1

In [2]:
class Error:
    """ Error function class"""
    
    @staticmethod
    def evaluate(u, v):
        """Evaluate the function
        Args:
        u (float): u coordinate
        v (float): v coordinate
        Returns:
        The function value (float)
        """
        return (u*exp(v)-2*v*exp(-u))**2
    
    @staticmethod
    def dEdu(u, v):
        """Evaluate the partial derivative dE/du of the function
        Args:
        u (float): u coordinate
        v (float): v coordinate
        Returns:
        The partial derivative dE/du (float)
        """
        return 2*(u*exp(v)-2*v*exp(-u))*(exp(v)+2*v*exp(-u))
    
    @staticmethod
    def dEdv(u, v):
        """Evaluate the partial derivative dE/dv of the function
        Args:
        u (float): u coordinate
        v (float): v coordinate
        Returns:
        The partial derivative dE/dv (float)
        """
        return 2*(u*exp(v)-2*v*exp(-u))*(u*exp(v)-2*exp(-u))
    
    @staticmethod
    def gradient(u, v):
        """Evaluate the gradient of the function
        Args:
        u (float): u coordinate
        v (float): v coordinate
        Returns:
        The function gradient value (float)
        """
        return np.array([Error.dEdu(u,v), Error.dEdv(u,v)])

In [3]:
class GradientDescent:
    """Gradient descent class"""
    
    @staticmethod
    def minimize(f, initial_guess=np.array([1.0, 1.0]), learning_rate = 0.1, conv_error=10**-14, max_iterations=20):
        """Perform gradient descent minimization
        Args:
        f (function): Error function to minimize
        initial_guess (numpy ndarray): Initial guess
        learning_rate (float): Learning rate
        conv_error (float): Error function value below which convergence is assumed 
        max_iterations (int): Maximum number of iterations
        Returns:
        Tuple(error, error_val, num_iterations) (Tuple(numpy ndarray, float, int): Tuple of error, 
        error function value at minimum, number of iterations
        """
        prev_error = np.array(initial_guess) 
        num_iterations = 0
        for i in range(0, max_iterations):
            error = prev_error - learning_rate * f.gradient(prev_error[0], prev_error[1])
            error_val = Error.evaluate(error[0], error[1])
            prev_error =  error
            num_iterations += 1
            if error_val < conv_error:
                break
        return error, error_val, num_iterations

In [4]:
error, error_val, num_iterations = GradientDescent.minimize(Error)
print('Final (u, v) = ', error)
print('Number of iterations needed = ', num_iterations)

Final (u, v) =  [0.04473629 0.02395871]
Number of iterations needed =  10


In [5]:
class CoordinateDescent:
    """Coordinate descent class"""
    
    @staticmethod
    def minimize(f, initial_guess=np.array([1.0, 1.0]), learning_rate = 0.1, max_iterations=15):
        """Perform coordinate descent minimization
        Args:
        f (function): Error function to minimize
        initial_guess (numpy ndarray): Initial guess
        learning_rate (float): Learning rate
        max_iterations (int): Maximum number of iterations
        Returns:
        Tuple(error, error_val) (Tuple(numpy ndarray, float): Tuple of error, error function value at minimum
        """
        prev_error = np.array(initial_guess) 
        error = np.zeros(2, dtype=float)
        for i in range(0, max_iterations):
            error[0] = prev_error[0] - learning_rate * f.dEdu(prev_error[0], prev_error[1]) # 1st step
            prev_error[0] = error[0]
            error[1] = prev_error[1] - learning_rate * f.dEdv(prev_error[0], prev_error[1]) # 2nd step
            prev_error[1] = error[1]
        error_val = Error.evaluate(error[0], error[1])
        return prev_error, error_val

In [6]:
error, error_val = CoordinateDescent.minimize(Error)
print('Final (u, v) = ', error)
print('Final error = ', error_val)

Final (u, v) =  [ 6.2970759  -2.85230695]
Final error =  0.13981379199615315


Logistic Regression

In [7]:
class Point:
    """ Point class for representing and manipulating random 2-dimensional coordinates. """
    
    def __init__(self, lower=-1.0, upper=1.0):
        """ Create a new random point assuming a uniform distribution between the upper and lower bounds. 
        Args:
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.lower = lower # lower bound
        self.upper = upper # upper bound
        self.coordinates = np.random.uniform(lower, upper, 2) # point (numpy.ndarray) (half open interval!)

In [8]:
class TargetFunction:
    """TargetFunction class for representing and manipulation random lines in 2-dimensional coordinates."""
    def __init__(self, lower=-1.0, upper=1.0):
        """ Create a new random line assuming a uniform distribution between the upper and lower bounds. 
        Args:
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.points = [Point(lower, upper) for i in range(0,2)]
        
    def sign(self, point):
        return np.sign(self.evaluate(point))
        
    def evaluate(self, point): 
        x_minus_x1  = point.coordinates[0] - self.points[0].coordinates[0]
        y_minus_y1  = point.coordinates[1] - self.points[0].coordinates[1]
        y2_minus_y1 = self.points[1].coordinates[1] - self.points[0].coordinates[1]
        x2_minus_x1 = self.points[1].coordinates[0] - self.points[0].coordinates[0]
        return x_minus_x1 * y2_minus_y1 - y_minus_y1 * x2_minus_x1
    
    def weights(self):
        y1_times_x2 = self.points[0].coordinates[1] * self.points[1].coordinates[0]
        x1_times_y2 = self.points[0].coordinates[0] * self.points[1].coordinates[1]
        y2_minus_y1 = self.points[1].coordinates[1] - self.points[0].coordinates[1]
        x2_minus_x1 = self.points[1].coordinates[0] - self.points[0].coordinates[0]   
        return np.array([y1_times_x2 - x1_times_y2, y2_minus_y1, -x2_minus_x1])

In [9]:
class DataSet:
    """ DataSet class. """
    def __init__(self, N=2, lower=-1.0, upper=1.0):
        """ Create a new data set assuming a uniform distribution between the 
            upper and lower bounds. 
        Args:
        N (int): Number of points.
        lower (float): Lower bound.
        upper (float): Upper bound.
        """
        self.inputs = [Point(lower, upper) for i in range(0,N)]

In [10]:
class LogisticRegression:
    """ Logistic Regression class. """
    
    def __init__(self, training_set):
        """ Create a new Logistic Regression. 
        Args:
        training_set (DataSet): Training set.
        """
        self.training_set = training_set
        
    def run(self, target_function, initial_weights=np.zeros(3, dtype=float), learning_rate=0.01, tol=0.01, max_epochs=1000):    
        """Run of Logistic Regression Algorithm using Stochstic Gradient Descent (SGD) to minimize the in-sample error.
        Args:
        target_function (TargetFunction): Target function
        initial_weights (np.ndarray): Initial guess for weight vector
        learning_rate (float): Learning rate
        tol (float): Tolerance in weight vector for which convergence is assumed
        max_epochs (int): Maximum number of epochs
        Returns:
        Tuple(g, num_epochs)
        """
        # set inputs and outputs
        x = self.transformPoints(self.training_set.inputs) # transform inputs
        y = np.array([target_function.sign(input) for input in self.training_set.inputs], copy=True)
        data = list(zip(x, y))
        
        # initialize weight vector
        w_prev_epoch = np.array(initial_weights)
        w_prev = w_prev_epoch
        
        num_epochs = 0
        for t in range(0, max_epochs):   
            num_epochs += 1
            # randomly permute the data points
            data_epoch = np.random.permutation(data)
            # use Stochastic Gradient Descent (SGD) to update the weight vector for each training example 
            for example in data_epoch:
                xn = example[0]
                yn = example[1]
                gradient_Ein = -yn * xn / (1. + np.exp(yn * np.dot(w_prev.T, xn)))
                w = w_prev - learning_rate * gradient_Ein
                w_prev = w    
            
            w_epoch = w
            #print('norm(w_prev_epoch - w_epoch) = ', norm(w_prev_epoch - w_epoch))
            if norm(w_prev_epoch - w_epoch) < tol:
                break
            w_prev_epoch = w_epoch
            
        return w_epoch, num_epochs
    
    def Eout(self, weights, target_function, n_points=1000):
        """Evaluate the cross-entropy error (Eout)
        Args:
        weights (np.ndarray): g function
        target_function (TargetFunction): Target function
        n_points (int): Number of points to generate
        Returns:
        Cross-entropy error (Eout)
        """
        # generate random points    
        estPoints = [Point(-1.0, 1.0) for i in range(0, n_points)]
        
        # compute inputs and outputs
        x = self.transformPoints(estPoints)
        y = np.array([target_function.sign(estPoint) for estPoint in estPoints], copy=True)
        
        # compute Eout
        eouts = []
        for i in range(0, len(x)):
            xn = x[i]
            yn = y[i]
            eouts.append(np.log(1. + np.exp(-yn * np.dot(weights.T, xn))))
        eout = np.mean(eouts)
        return eout
         
    # privates
    def transformPoints(self, points):
        return np.array([np.insert(pnt.coordinates, 0, 1.0) for pnt in points], copy=True)

In [11]:
# Create training set
N = 100
training_set = DataSet(N)

# Run logistic regression
num_runs = 100
eouts = []
n_epochs = []
for i in range(0, num_runs):
    lr = LogisticRegression(training_set) 
    target_function = TargetFunction()
    g, num_epochs = lr.run(target_function)
    eouts.append(lr.Eout(g, target_function))
    n_epochs.append(num_epochs)

eout = np.mean(eouts)
num_epoch = np.mean(n_epochs)
print('Eout = ', eout)
print('Number of epochs for convergence = ', num_epoch)

Eout =  0.10345886899317966
Number of epochs for convergence =  341.92
